# กระบวนการมอดูเลทและดีมอดูเลทสัญญาณ QAM
QAM เป็นการนำเอา ASK และ PSK มาผสมผสานกันเพื่อเพิ่มจำนวนรูปแบบของสัญญาณ ทำให้สัญญาณแต่ละรูปแบบแทนจำนวนบิตที่มากขึ้น ซึ่งมีผลทำให้ได้อัตราการเปลี่ยนแปลงสัญญาณ (baud rate) ที่ลดลง โดยคงอัตราการส่งข้อมูล (bit rate) ไว้เท่าเดิม

การสร้างสัญญาณที่มีรูปแบบของแอมพลิจูดและเฟสที่หลากหลายทำได้โดยการผสมคลื่นซายน์ 2 รูปคลื่นที่มีเฟส 0 องศา (in-phase component) และเฟส 90 องศา (quadrature component) ด้วยแอมพลิจูดที่แตกต่างกัน

## วัตถุประสงค์
* เข้าใจรายละเอียดของกระบวนการมอดูเลทสัญญาณด้วยกลไก QAM
* เข้าใจกระบวนการดีมอดูเลทสัญญาณที่ผ่านการมอดูเลทด้วยกลไก QAM และทราบถึงวิธีการตีความสัญญาณให้กลับมาในรูปข้อมูลดิจิทัลได้

## การเตรียมตัว
เรียกใช้งานโมดูลที่เกี่ยวข้อง โมดูลที่สำคัญในที่นี้คือโมดูล <tt>sigproc.analog</tt> ที่มีคลาสชื่อ <tt>Qam</tt> สำหรับสร้างสัญญาณด้วยกลไก QAM

In [1]:
import numpy as np
from sigproc import Signal,start_notebook
from sigproc.analog import Qam

bkp = start_notebook()

# สร้างการตั้งค่าการแสดงผลกราฟเอาไว้ใช้ภายหลัง
FOPTS = dict(height=250,y_range=(-1.2,1.2))

Loading BokehJS ...

Notebook session ready.


## สร้างสัญญาณ QAM ตัวอย่าง

In [3]:
BITRATE = 10
FC = 50
DATA = "00 01 10 11"
modulation = { 
    '00' : (1,-135),
    '01' : (1, 135),
    '10' : (1,-45),
    '11' : (1, 45),
    }

qam = Qam(modulation=modulation,bitrate=BITRATE,fc=FC)

รูปแบบสัญญาณที่มีแอมพลิจูดและเฟสที่แตกต่างกันนี้สามารถนำมาสร้างเป็นแผนภาพที่เรียกว่า constellation diagram เพื่อแสดงแอพลิจูดและเฟสของสัญญาณที่นำมาใช้แทนแพทเทิร์นบิตข้อมูลแต่ละแพทเทิร์นได้ ซึ่งทำได้โดยการเรียกใช้เมท็อด `plot_constellation()` ที่เตรียมไว้ในคลาส `Qam`

In [4]:
qam.plot_constellation()

คลาส <tt>Qam</tt> มีเมท็อด <tt>generate_signal()</tt> เพื่อสร้างสัญญาณแบบแอนะล็อกจากบิตข้อมูลที่ระบุ ตามรูปแบบ modulation ที่กำหนดเอาไว้ก่อนหน้านี้

In [5]:
s = qam.generate_signal(DATA)
s.plot_time(fig_options=FOPTS)

/Volumes/Data/cpe31/3/3:1/datacom/3/act6-analog/sigproc/sigproc.py:378: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  ts.loc[interval,"a"] = func(ts[interval].t.as_matrix()-offset)


## กระบวนการส่งสัญญาณ
เมท็อด `generate_signal()` ในคลาส `Qam` ซ่อนรายละเอียดที่สำคัญเอาไว้หลายส่วนเนื่องจากต้องการให้ทดลองสร้างสัญญาณได้โดยง่าย
กระบวนการส่งสัญญาณที่ครบถ้วนประกอบไปด้วยกลไกดังภาพ

![modulation](pics/modulation.png)

ต่อไปนี้จะแสดงการสร้างสัญญาณทีละขั้นตอนเพื่อให้ได้สัญญาณสุดท้ายตามกราฟข้างต้น

### สร้างสัญญาณ baseband
ใช้เมท็อด `baseband_signals()` ในคลาส `Qam` เพื่อสร้างสัญญาณ baseband 2 สัญญาณ (I และ Q) จากข้อมูลดิจิทัลที่ระบุ โดยดึงแอมพลิจูดของ I และ Q มาจาก constellation mapping 

In [6]:
baseband_i,baseband_q = qam.baseband_signals(DATA)
qam.plot_constellation()
print(f"DATA = {DATA}")

print("Baseband signal: I component")
baseband_i.plot_time(fig_options=FOPTS)
print("Baseband signal: Q component")
baseband_q.plot_time(fig_options=FOPTS)

DATA = 00 01 10 11
Baseband signal: I component


Baseband signal: Q component


### มอดูเลทสัญญาณลงไปบนคลื่นพาหะ
สัญญาณ baseband แต่ละองค์ประกอบ I และ Q จะถูกนำมาคูณเข้ากับคลื่นพาหะตามความถี่ที่กำหนดไว้ตั้งแต่แรก โดยคลื่นพาหะขององค์ประกอบ I และ Q จะมีเฟสที่แตกต่างกันอยู่ 90 องศา

เพื่อความสะดวก คลาส `Qam` มีสแตติกเมท็อด `generate_frequency()` ที่ใช้สร้างสัญญาณสำหรับคลื่นพาหะเพื่อนำไปคูณกับสัญญาณ baseband ข้างต้นได้ทันที

In [ ]:
carrier_i = Qam.generate_frequency(FC,baseband_i.duration,phase=0)
carrier_q = Qam.generate_frequency(FC,baseband_i.duration,phase=90)
mod_i = carrier_i*baseband_i
mod_q = carrier_q*baseband_q
print("Modulated I component")
mod_i.plot_time(fig_options=FOPTS)
print("Modulated Q component")
mod_q.plot_time(fig_options=FOPTS)

### ผสมสัญญาณและส่งออก
เมื่อนำสัญญาณที่มอดูเลทแล้วทั้งองค์ประกอบ I และ Q มาผสมกัน จะได้ผลลัพธ์เป็นสัญญาณของคลื่นพาหะที่มีเฟสชิฟต์เป็นไปตามที่กำหนดเอาไว้ใน constellation mapping ตั้งแต่แรก เห็นได้ว่าผลลัพธ์เหมือนกับสัญญาณที่สร้างจากเมท็อด `generate_signal` ข้างต้นทุกประการ

In [ ]:
mod_signal = mod_i + mod_q
mod_signal.plot_time(fig_options=FOPTS)

## กระบวนการรับสัญญาณ
ส่วนแรกของกระบวนการรับสัญญาณคือการดีมอดูเลทสัญญาณให้ได้สัญญาณ baseband กลับออกมา หลักการในการดีมอดูเลทสัญญาณอาศัยเอกลักษณ์พื้นฐานทางตรีโกณดังนี้

$$
  \sin^2 x = \frac{1-\cos 2x}{2},~~
  \cos^2 x = \frac{1+\cos 2x}{2}~~และ~~
  \sin x\cdot\cos x = \frac{\sin 2x}{2}
$$

สมมติให้ฟังก์ชัน $I(t)$ และ $Q(t)$ แทนสัญญาณ baseband ในองค์ประกอบ I และ Q ตามลำดับ สัญญาณผลลัพธ์ $s(t)$ หลังจากผ่านกระบวนการมอดูเลชันคือ

$$ s(t) = I(t)\sin 2\pi f_c t + Q(t)\cos 2\pi f_c t $$

ฝั่งรับสัญญาณต้องการแยกสัญญาณ baseband ออกจากคลื่นพาหะ ด้วยหลักการตรีโกณข้างต้นจะเห็นว่าเมื่อนำเอาสัญญาณ $s(t)$ มาคูณกับคลื่นพาหะ $\sin 2\pi f_c t$ จะทำให้สามารถแยกพจน์ของ $I(t)$ออกมาได้ดังนี้

$$
\begin{eqnarray*}
  s(t)\cdot\sin 2\pi f_c t
    &=& I(t)\sin^2 2\pi f_c t + Q(t)\cos 2\pi f_c t \cdot\sin 2\pi f_c t \\
    &=& \frac{1}{2}I(t) - \frac{1}{2}I(t)\cos 4\pi f_c t + \frac{1}{2}Q(t)\sin 4\pi f_c t
\end{eqnarray*}
$$

เห็นได้ว่าพจน์ $-\frac{1}{2}I(t)\cos 4\pi f_c t$ และ $\frac{1}{2}Q(t)\sin 4\pi f_c t$ ให้ความถี่กระจายอยู่รอบบริเวณ $2f_c$ ดังนั้นเมื่อนำผลคูณของสัญญาณข้างต้นมากรองด้วย low-pass filter และคูณด้วยสอง จะเหลือเพียงสัญญาณที่ประมาณค่าใกล้เคียงกับสัญญาณ baseband $I(t)$ ดั้งเดิม

ในทำนองเดียวกัน เมื่อนำผลคูณ $s(t)\cdot \cos 2\pi f_c t$ มากรองด้วย low-pass filter และคูณด้วยสอง จะได้สัญญาณที่ใกล้เคียงกับสัญญาณ baseband $Q(t)$ ดั้งเดิมเช่นเดียวกัน

$$
\begin{eqnarray*}
  s(t)\cdot\cos 2\pi f_c t
    &=& I(t)\sin 2\pi f_c t \cdot \cos 2\pi f_c t + Q(t)\cos^2 2\pi f_c t \\
    &=& \frac{1}{2}I(t)\sin 4\pi f_c t + \frac{1}{2}Q(t) + \frac{1}{2}Q(t)\cos 4\pi f_c t
\end{eqnarray*}
$$

สัญญาณ $I(t)$ และ $Q(t)$ ที่แยกออกมาได้จะถูกนำไปเปรียบเทียบกับ constellation เพื่อตรวจหาซิมโบลที่ใกล้เคียงที่สุดและตีความเป็นข้อมูลดิจิทัลต่อไป

กระบวนการดีมอดูเลทสัญญาณทั้งหมดจึงมีขั้นตอนดังภาพ

![demodulation](pics/demodulation.png)

### ดีมอดูเลทสัญญาณ
โค้ดด้านล่างใช้กระบวนการข้างต้นดีมอดูเลทสัญญาณเพื่อให้ได้สัญญาณ baseband ทั้ง I และ Q

In [ ]:
rx_i = (mod_signal*carrier_i*2).filter(FC)
rx_q = (mod_signal*carrier_q*2).filter(FC)

print("Demodulated baseband signal: I component")
rx_i.plot_time(fig_options=FOPTS)
print("Demodulated baseband signal: Q component")
rx_q.plot_time(fig_options=FOPTS)

### ตีความสัญญาณ baseband
ขั้นตอนสุดท้ายของกระบวนการรับสัญญาณคือนำสัญญาณ baseband ที่ได้กลับคืนมาไปเปรียบเทียบกับ constellation ที่กำหนด โค้ดด้านล่างแสดงการสุ่มค่าสัญญาณจากจุดกึ่งกลางของแต่ละซิมโบลพร้อมแปลงค่าในรูปแบบ in-phase/quadrature ให้เป็นรูปแบบ magnitude/phase เพื่อให้เปรียบเทียบกับตารางกำหนดรูปสัญญาณข้างต้นได้สะดวก

In [ ]:
# คำนวณหาระยะเวลาของ 1 symbol
symbol_interval = 1/qam.baud_rate

# กำหนดจุดเวลาที่ใช้สุ่มสัญญาณ โดยสุ่ม ณ ตำแหน่งกึ่งกลางของซิมโบล
sampling = np.arange(0,mod_signal.duration,symbol_interval) + symbol_interval/2

# พล็อตจุดแสดงตำแหน่งที่สุ่มตรวจสอบค่าในสัญญาณ
fig = bkp.figure(**FOPTS)
rx_i.plot_time(fig=fig,line_options=dict(color="red",legend="I"))
rx_q.plot_time(fig=fig,line_options=dict(color="blue",legend="Q"))
fig.scatter(sampling*1000,rx_i.time_function(sampling),line_color="red",fill_color="yellow",size=10)
fig.scatter(sampling*1000,rx_q.time_function(sampling),line_color="blue",fill_color="yellow",size=10)
bkp.show(fig)

โค้ดด้านล่างแสดงค่าของแต่ละซิมโบลที่อ่านมาได้ โดยแสดงทั้งในรูป in-phase/quadrature และในรูป magnitude/phase (การคำนวณอาศัยคณิตศาสตร์ของจำนวนเชิงซ้อนแทนการใช้ตรีโกณ เนื่องจากไพทอนเตรียมฟังก์ชันสำหรับจำนวนเชิงซ้อนที่พร้อมนำมาใช้งานได้ง่าย)

In [ ]:
points = rx_i.time_function(sampling)+rx_q.time_function(sampling)*1j
for i,z in enumerate(points):
    mag = np.abs(z)
    phase = np.degrees(np.angle(z))
    print(f"Symbol #{i+1}: (I={z.real:4.1f}, Q={z.imag:4.1f}) -> (Mag={mag:.1f}, Phase={phase:6.1f})")